# Single Training on GPU 0

# Quick Links <a name = "Top"></a>

<ol>
<li><a href = #setup>Begin Training</a></li>
</ol>

# Imports

In [1]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"]='0'

print('Current Conda Environment: {}'.format(os.environ['CONDA_DEFAULT_ENV']))

Current Conda Environment: tf_ks


In [2]:
import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import tensorflow as tf
  
available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad
from tensorflow.keras.layers import ReLU, LeakyReLU

from datetime import datetime
import pandas as pd
import numpy as np
import shutil

from copy import deepcopy

import time

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  2 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7606523854522899611
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9105744200
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1914957022813141045
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 9104897474
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10480317982496326279
physical_device_desc: "device: 1, name: GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5"
]


# Hilfsfunktionen

### Enum für Training-Set

In [3]:
from enum import Enum

class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Output Directory

* <i>SSD</i>, falls genug Speicher auf SSD im SymLink <i>fast_output</i> verfügbar ist
* <i>HDD</i>, falls möglicherweise zu wenig SSD-Speicher verfügbar ist $\rightarrow$ <i>output</i>

In [4]:
from enum import IntEnum

class OutputDirectory(IntEnum):
    HDD = 0
    SSD = 1
    
output_path = ['output', 'fast_output']

### Benutzerdefinierte Kostenfunktion & Metrik

In [5]:
def circular_mse(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.square(K.minimum(K.abs(y_pred - y_true), max_error - K.abs(y_pred - y_true))), axis = -1)

def circular_mae(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.minimum(K.abs(y_pred - y_true), K.abs(max_error - K.abs(y_pred - y_true))), axis = -1)

def custom_mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Convert Label_Type into suitable label names.
$\Rightarrow$ Angular / Normalized $\rightarrow$ ['Elevation', 'Azimuth']

$\Rightarrow$ Stereographic $\rightarrow$ ['S_x', 'S_y']

In [6]:
def get_Label_Names(label_type):
    if label_type == 'Angular' or label_type == 'Normalized':
        return ['Elevation', 'Azimuth']
    elif label_type == 'Stereographic':
        return ['S_x', 'S_y']
    else:
        assert(True, 'LabelType Invalid')
        return None

### Convert String into Reduction Metric Function

In [7]:
def get_Reduction_Metric(metric):
    
    if metric == 'custom_mae':
        return [custom_mae]
    elif metric == 'tf.keras.metrics.MeanAbsoluteError()':
        return [tf.keras.metrics.MeanAbsoluteError()]
    elif metric == 'circular_mae':
        return [circular_mae]
    elif metric == 'mean_squared_error':
        return ['mean_squared_error']
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Automatische Optimizer Generierung aus String

In [8]:
def make_optimizer(optimizer):
    # [Adam, Nadam, Adagrad, RMSprop]
    if optimizer == "<class 'keras.optimizers.Adam'>":
        return Adam
    elif optimizer == "<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>":
        return Adam
    elif optimizer == "<class 'keras.optimizers.Nadam'>":
        return Nadam
    elif optimizer == "<class 'keras.optimizers.Adagard'>":
        return Adagard
    elif optimizer == "<class 'keras.optimizers.RMSprop'>":
        return RMSprop
    else:
        print('ERROR::: Unspecified Optimizer')

### Trainingsset-Typ nach String Converter

In [9]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

### Generierung Datenpipeline (Angepasst für Talos)

In [10]:
def create_data(batch_size, num_samples, label_type):
    # if Block für synthetische Daten, um nur auf realen Daten zu trainieren _USE_SYNTHETIC_TRAIN_DATA
    # 1. lege df_train und df_valid als leere Liste an
    # 2. If-block um Zeile df = ... bis df_valid
    
    if trainingset == TrainingSet.SYNTHETIC:
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
        df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
        
    elif trainingset == TrainingSet.MIXED:
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
        df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
        
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_shuffled_real = df_shuffled_real.drop(df_shuffled_real.index[(df_shuffled_real.shape[0] - 61) : df_shuffled_real.shape[0]])
        df_train_real = df_shuffled_real[0: int(df_shuffled_real.shape[0] * 0.8 // batch_size * batch_size)]   
        df_valid_real = df_shuffled_real.drop(df_shuffled_real.index[0 : df_train_real.shape[0]])
        df_train = df_train.drop(df_train.index[df_train.shape[0] - df_train_real.shape[0] : df_train.shape[0]])
        df_valid = df_valid.drop(df_valid.index[df_valid.shape[0] - df_valid_real.shape[0] : df_valid.shape[0]])
        df_train = df_train.append(df_train_real)
        df_valid= df_valid.append(df_valid_real)
    
    elif trainingset == TrainingSet.REAL: # Add check for num_samples, once the real dataset increases
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_shuffled_real = df_shuffled_real.drop(df_shuffled_real.index[(df_shuffled_real.shape[0] - 61) : df_shuffled_real.shape[0]])
        df_train = df_shuffled_real[0 : int(df_shuffled_real.shape[0] * 0.8 // batch_size * batch_size)]   
        df_valid = df_shuffled_real.drop(df_shuffled_real.index[0 : df_train.shape[0]])
        
    else:
        print('Create_Data :: should not have reached here')
        

        
    if _USE_DATA_AUGMENTATION:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.1,
            brightness_range = (0.25, 0.75),
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255
        )
        
    print('Y-Col: {}'.format(get_Label_Names(label_type)))
    print('Train Data Generator: ', end = '')
    
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = _IMAGE_DIR,
        x_col = 'Filename',
        y_col = get_Label_Names(label_type),
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = 'rgb',
        shuffle = True,
        seed = 77,
        batch_size = batch_size
    )
    
    valid_data_generator = ImageDataGenerator(
        rescale = 1./255
    )
    
    print('Validation Data Generator: ', end = '')
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = _IMAGE_DIR,
        x_col = 'Filename',
        y_col = get_Label_Names(label_type),
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = 'rgb',
        shuffle = False,
        seed = 77,
        batch_size = batch_size
    )
    
    return train_generator, valid_generator

### Generierung Modell (Angepasst für Talos)

In [11]:
def grid_model_fine(x, y, x_val, y_val, params):
    
    K.clear_session()
    
    train_generator, valid_generator = create_data(params['batch_size'], params['samples'], params['label_type'])
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
    dropout_rate = params['dropout']
    first_neuron = params['first_neuron']
    
    if params['activation'] == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = params['leaky_alpha'])
    elif params['activation'] == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))
    
    for layer in cnn.layers[:15]:
        layer.trainable = False
        #print(layer.name, layer.trainable)
        
    print('_________________________________________________________________')
    print('{:>16} {:>16}'.format('Network Layer', 'Trainable'))
    print('=================================================================')
    for layer in cnn.layers:
        print('{:>16} {:>16}'.format(layer.name, layer.trainable))
    print('_________________________________________________________________\n')
    
    model.add(cnn)
    
    fc = Sequential()
    fc.add(Flatten(input_shape = model.output_shape[1:])) # (7, 7, 512)
    
    fc.add(Dense(units = first_neuron, kernel_initializer = glorot_uniform(seed = 1)))
    fc.add(activation_layer)
    if dropout_rate > 0.0:
        fc.add(Dropout(rate = dropout_rate))
    
    print('Number Hidden Layers {}'.format(params['hidden_layers']))
    hidden_neuron_fraction = first_neuron
    for i in range(params['hidden_layers']):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        fc.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
        fc.add(activation_layer)
        if dropout_rate > 0.0:
            fc.add(Dropout(rate = dropout_rate))
    
    fc.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))
    fc.load_weights(_MODEL_DIR + _MODEL_TO_LOAD)
    model.add(fc)
    print('Fully Connected Layers added to Base Network')
    
    print('Using Loss: {} \nand Reduction Metric: {}'.format(
        params['loss_function'], 
        get_Reduction_Metric(params['reduction_metric'])))
    
    model.compile(
        #optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])*1e-2),
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer']) * 1e-3),
        loss = params['loss_function'],
        metrics = get_Reduction_Metric(params['reduction_metric'])
    )
    print('Model was compiled')
    print(model.summary())
    print('_________________________________________________________________')
    
    checkpointer = callbacks.ModelCheckpoint(
        filepath = _LOG_DIR + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(_MODEL_TO_LOAD_INDEX, train_generator.n),
        monitor =  params['monitor_value'],
        verbose = 1,
        save_weights_only = False,
        save_best_only = True,
        mode = 'min'
    )
    print('Checkpointer was created')
    
    csv_logger = callbacks.CSVLogger(
        filename = _LOG_DIR + 'CNN_Base_{}_Logger_{}.csv'.format(_MODEL_TO_LOAD_INDEX, train_generator.n),
        separator = ',',
        append = False
    )
    print('CSV Logger was created')

    lr_reducer = callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = 0.1,
        patience = 13,
        verbose = 1,
        mode = 'min',
        min_delta = 0.0001
    )
    print('Learning Rate Reducer was created')
    
    early_stopper = callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0,
        #patience = 15,
        patience = 20,
        verbose = 1,
        mode = 'min',
        restore_best_weights = True
    )
    print('Early Stopper was created')
    
    out = model.fit(
        x = train_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_data = valid_generator,
        validation_steps = vg_validation_steps,
        callbacks = [checkpointer, csv_logger, lr_reducer, early_stopper],
        epochs = params['epochs'],
        workers = 8
    )
    
    return out, model

# Feinoptimierung <a name = "setup"></a><a href = #Top>Up</a></p>

#### Hyper Parameter

In [12]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad

global_hyper_parameter = {
    'samples': None,
    'epochs': None,
    'batch_size': None,
    'optimizer': None,
    'lr': None,
    'first_neuron': None,
    'dropout': None,
    'activation': None,
    'leaky_alpha': None,
    'hidden_layers': None,
    # beginning from here, Values should only contain one single entry:
    # ===============================================================
    'label_type': ['Normalized'], # Stereographic, Angular, Normalized
    'loss_function': None,
    'reduction_metric': None,
    'monitor_value': None
}


### Training Setup 

In [13]:
_RUN = 'SYNTH'
_LOSS = 'MSE'
_DATASET_NAME = '2020-05-28'
_DEVICE = 'TITAN_GPU0'

storage = OutputDirectory.SSD # 'fast_output' if ssd storage may suffice, 'output' otherwise

if global_hyper_parameter['label_type'][0] == 'Stereographic':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch_stereographic.csv'
    _CSV_REAL_FILE_NAME = 'images_real_stereographic.csv'
    
elif global_hyper_parameter['label_type'][0] == 'Angular':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch.csv'
    _CSV_REAL_FILE_NAME = 'images_real.csv'
    
elif global_hyper_parameter['label_type'][0] == 'Normalized':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch_normalized.csv'
    _CSV_REAL_FILE_NAME = 'images_real_normalized.csv'
    
else:
    assert(True, 'Label Type Invalid')

In [14]:
trainingset = TrainingSet.SYNTHETIC
_USE_DATA_AUGMENTATION = True

In [15]:
_IMAGE_DIR = '..\\dataset_mm\\{}\\'.format(_DATASET_NAME)
_CSV_FILE = _IMAGE_DIR + _CSV_SYNTH_FILE_NAME
_CSV_FILE_REAL = _IMAGE_DIR + _CSV_REAL_FILE_NAME

_note = '_Custom-MAE'

_MODEL_DIR = '..\\output\\{}_Regression_{}\\{}_{}_Base{}\\'.format(_RUN, _LOSS, _DATASET_NAME, global_hyper_parameter['label_type'][0], _note)
_NET_DIR = '{}_Regression_{}\\{}_{}_Top_1{}\\{}_TD\\'.format(_RUN, _LOSS, _DATASET_NAME, global_hyper_parameter['label_type'][0], _note, trainingset_to_string(trainingset))
_LOG_DIR = '..\\{}\\{}'.format(output_path[storage], _NET_DIR)

if(not os.path.exists(_LOG_DIR)):
    os.makedirs(_LOG_DIR)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(_LOG_DIR))

device_file = open(_LOG_DIR + '{}.txt'.format(_DEVICE), "a+")

Directory >>| ..\fast_output\SYNTH_Regression_MSE\2020-05-28_Normalized_Top_1_Custom-MAE\Synth_TD\ |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)


### Top 3 FC-Gewichte

In [16]:
base_results = _MODEL_DIR + '..\\{}_{}_Base{}_Results.csv'.format(_DATASET_NAME, global_hyper_parameter['label_type'][0], _note)
df = pd.read_csv(base_results).drop(columns = ['round_epochs', 'samples', 'epochs'], axis = 0)
sort_value = df['monitor_value'][0]
df = df.sort_values(sort_value, axis = 0, ascending = True, inplace = False, kind = 'quicksort', na_position = 'last')
print('Displaying: {}'.format(base_results))
df.head(10)

Displaying: ..\output\SYNTH_Regression_MSE\2020-05-28_Normalized_Base_Custom-MAE\..\2020-05-28_Normalized_Base_Custom-MAE_Results.csv


,Unnamed: 0,start,end,duration,loss,custom_mae,val_loss,val_custom_mae,activation,batch_size,dropout,first_neuron,hidden_layers,label_type,loss_function,lr,monitor_value,optimizer,reduction_metric
210,210,10/29/20-193146,10/29/20-193201,14.936548,10.795315,0.595088,0.101124,0.214072,relu,32,0.25,4096,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
195,195,10/29/20-192905,10/29/20-192915,10.025699,5.247484,0.605640,0.100757,0.215078,relu,32,0.25,2048,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
180,180,10/29/20-192658,10/29/20-192709,11.400928,2.732533,0.477908,0.104949,0.220771,relu,32,0.25,1024,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
300,300,10/29/20-194850,10/29/20-194902,12.204615,22.274675,0.953535,0.112903,0.238330,relu,64,0.25,4096,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
211,211,10/29/20-193201,10/29/20-193216,14.956688,42.028975,0.893910,0.116852,0.238365,relu,32,0.25,4096,0,Normalized,mean_squared_error,2,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
240,240,10/29/20-193755,10/29/20-193805,10.143713,4.793068,0.598642,0.116115,0.239593,relu,32,0.50,2048,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
183,183,10/29/20-192725,10/29/20-192733,7.764266,1.332591,0.489436,0.118337,0.241562,relu,32,0.25,1024,1,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
285,285,10/29/20-194634,10/29/20-194643,8.401402,10.567411,0.992507,0.117695,0.242329,relu,64,0.25,2048,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
196,196,10/29/20-192915,10/29/20-192925,9.898231,18.965731,0.943573,0.118273,0.242819,relu,32,0.25,2048,0,Normalized,mean_squared_error,2,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
345,345,10/29/20-195613,10/29/20-195625,12.194129,21.463338,0.990857,0.118296,0.245393,relu,64,0.50,4096,0,Normalized,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae


### GridSerach

In [17]:
def get_params(top_results_index):
    
    #     Adam = RMSprop + Momentum (lr=0.001)
    #     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
    #     RMSprop = (lr=0.001)
    #     SGD = (lr=0.01)
    #     Adagrad

    hyper_parameter = global_hyper_parameter

    hyper_parameter['samples'] = [100000]
    hyper_parameter['epochs'] = [400]
    hyper_parameter['batch_size'] = [df.iloc[top_results_index]['batch_size']]
    hyper_parameter['optimizer'] = [make_optimizer(df.loc[top_results_index]['optimizer'])]
    hyper_parameter['lr'] = [df.iloc[top_results_index]['lr']]
    hyper_parameter['first_neuron'] = [df.iloc[top_results_index]['first_neuron']]
    hyper_parameter['dropout'] = [df.iloc[top_results_index]['dropout']]
    hyper_parameter['activation'] = [df.iloc[top_results_index]['activation']]
    hyper_parameter['leaky_alpha'] = [0.1] #Default bei LeakyReLU, sonst PReLU
    hyper_parameter['hidden_layers'] = [df.iloc[top_results_index]['hidden_layers']]
    
    hyper_parameter['loss_function'] = [df.iloc[top_results_index]['loss_function']]
    hyper_parameter['reduction_metric'] = [df.iloc[top_results_index]['reduction_metric']]
    hyper_parameter['monitor_value'] = [df.iloc[top_results_index]['monitor_value']]

    return hyper_parameter

### Start Talos

In [18]:
dummy_x = np.empty((1, 2, 3, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    #for top_results_index in range(3):
    for top_results_index in [0, 1]:
        #top_results_index = 1
        _MODEL_TO_LOAD_INDEX = df.iloc[top_results_index].name
        _MODEL_TO_LOAD = 'Best_Weights_FC_{}.hdf5'.format(_MODEL_TO_LOAD_INDEX)

        _TMP_DIR = '..\\TMP_TALOS_{}'.format(_DEVICE)
        _CSV_RESULTS = _LOG_DIR + 'Talos_Results_Fine_Idx{}.csv'.format(_MODEL_TO_LOAD_INDEX)

        startTime = datetime.now()
        
        parameters = get_params(top_results_index)

        t = ta.Scan(
            x = dummy_x,
            y = dummy_y,
            model = grid_model_fine,
            params = parameters,
            experiment_name = _TMP_DIR,
            #shuffle=False,
            reduction_metric = parameters['reduction_metric'][0],
            disable_progress_bar = False,
            print_params = True,
            clear_session = True
        )

        print("Time taken:", datetime.now() - startTime)
        
        print('Writing Device File')
        device_file.write('Trained Model: {}'.format(_MODEL_TO_LOAD))

        df_experiment_results = pd.read_csv(_TMP_DIR + '\\' + os.listdir(_TMP_DIR)[0])
        df_experiment_results['Base'] = None
        for i in range(df_experiment_results.shape[0]):
            df_experiment_results['Base'][i] = _MODEL_TO_LOAD_INDEX

        if os.path.isfile(_CSV_RESULTS):
            df_experiment_results.to_csv(_CSV_RESULTS, mode = 'a', index = False, header = False)
        else:
            df_experiment_results.to_csv(_CSV_RESULTS, index = False)

        shutil.rmtree(_TMP_DIR)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

{'activation': 'relu', 'batch_size': 32, 'dropout': 0.25, 'epochs': 400, 'first_neuron': 4096, 'hidden_layers': 0, 'label_type': 'Normalized', 'leaky_alpha': 0.1, 'loss_function': 'mean_squared_error', 'lr': 1, 'monitor_value': 'val_custom_mae', 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'reduction_metric': 'custom_mae', 'samples': 100000}
Y-Col: ['Elevation', 'Azimuth']
Train Data Generator: Found 80000 validated image filenames.
Validation Data Generator: Found 20000 validated image filenames.
Steps per Epoch: 2500, Validation Steps: 625
_________________________________________________________________
   Network Layer        Trainable
         input_1                0
    block1_conv1                0
    block1_conv2                0
     block1_pool                0
    block2_conv1                0
    block2_conv2                0
     block2_pool                0
    block3_conv1                0
    block3_conv2                0
    block3_conv3    

2500/2500 [==============================] - 290s 116ms/step - loss: 0.0570 - custom_mae: 0.1488 - val_loss: 0.0492 - val_custom_mae: 0.1314
Epoch 14/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0554 - custom_mae: 0.1458
Epoch 00014: val_custom_mae improved from 0.13137 to 0.12808, saving model to ..\fast_output\SYNTH_Regression_MSE\2020-05-28_Normalized_Top_1_Custom-MAE\Synth_TD\CNN_Base_210_Model_and_Weights_80000.hdf5
2500/2500 [==============================] - 290s 116ms/step - loss: 0.0554 - custom_mae: 0.1458 - val_loss: 0.0477 - val_custom_mae: 0.1281
Epoch 15/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0542 - custom_mae: 0.1434
Epoch 00015: val_custom_mae improved from 0.12808 to 0.12795, saving model to ..\fast_output\SYNTH_Regression_MSE\2020-05-28_Normalized_Top_1_Custom-MAE\Synth_TD\CNN_Base_210_Model_and_Weights_80000.hdf5
2500/2500 [==============================] - 290s 116ms/step - loss: 0.0542 - custom_mae: 0.1434 - val_los

Epoch 54/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0328 - custom_mae: 0.1025
Epoch 00054: val_custom_mae improved from 0.09389 to 0.09353, saving model to ..\fast_output\SYNTH_Regression_MSE\2020-05-28_Normalized_Top_1_Custom-MAE\Synth_TD\CNN_Base_210_Model_and_Weights_80000.hdf5
2500/2500 [==============================] - 292s 117ms/step - loss: 0.0328 - custom_mae: 0.1025 - val_loss: 0.0320 - val_custom_mae: 0.0935
Epoch 55/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0325 - custom_mae: 0.1018
Epoch 00055: val_custom_mae did not improve from 0.09353
2500/2500 [==============================] - 287s 115ms/step - loss: 0.0324 - custom_mae: 0.1017 - val_loss: 0.0337 - val_custom_mae: 0.0962
Epoch 56/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0320 - custom_mae: 0.1009
Epoch 00056: val_custom_mae did not improve from 0.09353
2500/2500 [==============================] - 289s 115ms/step - loss: 0.0320 - custom_mae: 0.10

2499/2500 [============================>.] - ETA: 0s - loss: 0.0248 - custom_mae: 0.0866 ETA: 1s - l
Epoch 00099: val_custom_mae did not improve from 0.08129
2500/2500 [==============================] - 373s 149ms/step - loss: 0.0248 - custom_mae: 0.0866 - val_loss: 0.0269 - val_custom_mae: 0.0822
Epoch 100/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0248 - custom_mae: 0.0865
Epoch 00100: val_custom_mae improved from 0.08129 to 0.08087, saving model to ..\fast_output\SYNTH_Regression_MSE\2020-05-28_Normalized_Top_1_Custom-MAE\Synth_TD\CNN_Base_210_Model_and_Weights_80000.hdf5
2500/2500 [==============================] - 389s 156ms/step - loss: 0.0248 - custom_mae: 0.0865 - val_loss: 0.0257 - val_custom_mae: 0.0809
Epoch 101/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0246 - custom_mae: 0.0862
Epoch 00101: val_custom_mae did not improve from 0.08087
2500/2500 [==============================] - 377s 151ms/step - loss: 0.0246 - custom_mae: 0.0

Epoch 146/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0207 - custom_mae: 0.0778
Epoch 00146: val_custom_mae did not improve from 0.07285
2500/2500 [==============================] - 295s 118ms/step - loss: 0.0208 - custom_mae: 0.0778 - val_loss: 0.0238 - val_custom_mae: 0.0737
Epoch 147/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0206 - custom_mae: 0.0774
Epoch 00147: val_custom_mae did not improve from 0.07285
2500/2500 [==============================] - 294s 118ms/step - loss: 0.0206 - custom_mae: 0.0774 - val_loss: 0.0236 - val_custom_mae: 0.0752
Epoch 148/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0207 - custom_mae: 0.0778
Epoch 00148: val_custom_mae did not improve from 0.07285
2500/2500 [==============================] - 298s 119ms/step - loss: 0.0207 - custom_mae: 0.0778 - val_loss: 0.0229 - val_custom_mae: 0.0751
Epoch 149/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0204 - custom_mae: 0

2500/2500 [==============================] - 305s 122ms/step - loss: 0.0185 - custom_mae: 0.0741 - val_loss: 0.0216 - val_custom_mae: 0.0706
Epoch 171/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0187 - custom_mae: 0.0743
Epoch 00171: val_custom_mae did not improve from 0.07056
2500/2500 [==============================] - 297s 119ms/step - loss: 0.0187 - custom_mae: 0.0743 - val_loss: 0.0217 - val_custom_mae: 0.0706
Epoch 172/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0185 - custom_mae: 0.0741
Epoch 00172: val_custom_mae did not improve from 0.07056
2500/2500 [==============================] - 300s 120ms/step - loss: 0.0185 - custom_mae: 0.0741 - val_loss: 0.0217 - val_custom_mae: 0.0707
Epoch 173/400
2499/2500 [============================>.] - ETA: 0s - loss: 0.0188 - custom_mae: 0.0742
Epoch 00173: val_custom_mae did not improve from 0.07056
2500/2500 [==============================] - 296s 118ms/step - loss: 0.0188 - custom_mae: 0.0742 

MemoryError: Unable to allocate 392. MiB for an array with shape (25088, 4096) and data type float32

In [ ]:
device_file.close()

# Copy Results to NAS if SSD Directory was selected

In [ ]:
def copy_directory(src, dst, symlinks = False, ignore = None):
    maxLen = 0
    message = ''        
    
    if not os.path.exists(dst):
        
        message = 'Creating Path: {}'.format(src)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\r')
        
        os.makedirs(dst)
        
    for item in os.listdir(src):
        
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        
        if os.path.isdir(s):
            
            message = 'Copying Directory: {}'.format(s)
            maxLen = max(maxLen, len(message))
            print(message + ' ' * (maxLen - len(message)), end = '\r')
            
            shutil.copytree(s, d, symlinks, ignore)
            
        else:
            
            if not os.path.exists(d): #or os.stat(s).st_mtime - os.stat(d).st_mtime > 1:
                
                message = 'Copying File: {}'.format(s)
                maxLen = max(maxLen, len(message))
                print(message + ' ' * (maxLen - len(message)), end = '\r')
                
                shutil.copy2(s, d)
        
        time.sleep(.5)
     
    message = 'Coyping... Done'
    maxLen = max(maxLen, len(message))
    print(message + ' ' * (maxLen - len(message)), end = '\n')

def delete_directory(src, terminator = '\n'):
    message = ''
    maxLen = 0
    
    try:
        message = 'Deleting {}'.format(src)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\r')
        
        shutil.rmtree(src)
        
    except OSError as e:
        message = 'Error: {} : {}'.format(src, e.strerror)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\n')
        return
    
    message = 'Deleting... Done'
    maxLen = max(maxLen, len(message))
    print(message + ' ' * (maxLen - len(message)), end = terminator)

    
def copy_fine_training(src, dst):
    copy_directory(src, dst)
    delete_directory(src, terminator = '\r')
    delete_directory(src + '..\\', terminator = '\r')
    if not os.listdir(src + '..\\..\\'):
        delete_directory(src + '..\\..\\', terminator = '\r')

In [ ]:
if(storage == OutputDirectory.SSD):
    _COPY_DIR = '..\\output\\{}'.format(_NET_DIR)
    copy_fine_training(_LOG_DIR, _COPY_DIR)

# <a name = "CMSE.Mixed"></a><a href = #Top>Up</a></p>